#### Rex Gayas
#### Week 12 Exercise 12.2 Spring 2024
#### DSC400-T301 Big Data, Technology, and Algo (2245-1)
#### Deploying Machine Learning Models

In [1]:
from wsgiref.simple_server import make_server
import falcon
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing import image
import numpy as np
import io
import requests
from PIL import Image


In [3]:
class ClassifyResource:
    def __init__(self):
        self.model = ResNet50(weights='imagenet')

    def on_get(self, req, resp):
        img_url = req.get_param('img')
        if not img_url:
            resp.status = falcon.HTTP_BAD_REQUEST
            resp.media = {'error': 'Image URL is required'}
            return

        try:
            response = requests.get(img_url)
            img = Image.open(io.BytesIO(response.content))
            img = img.resize((224, 224))
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)

            predictions = self.model.predict(img_array)
            decoded_predictions = decode_predictions(predictions, top=3)[0]

            # Convert predictions to native Python data types
            result = [{'class': pred[1], 'description': pred[1], 'score': float(pred[2])} for pred in decoded_predictions]

            resp.media = {'predictions': result}
            resp.status = falcon.HTTP_OK
        except Exception as e:
            resp.status = falcon.HTTP_INTERNAL_SERVER_ERROR
            resp.media = {'error': str(e)}



In [4]:
app = falcon.App()
classify = ClassifyResource()
app.add_route('/classify', classify)


In [5]:
def run_server(app):
    with make_server('', 9999, app) as httpd:
        print('Serving on port 9999...')
        try:
            httpd.serve_forever()
        except KeyboardInterrupt:
            print('Stopping web server')

run_server(app)


Serving on port 9999...
1/1 [==============================] - 1s 783ms/step


127.0.0.1 - - [01/Jun/2024 02:17:52] "GET /classify?img=https%3A%2F%2Fs7d1.scene7.com%2Fis%2Fimage%2FCENODS%2F09928-leadcon-protein%3F%24responsive%24%26wid%3D700%26qlt%3D90%2C0%26resMode%3Dsharp2 HTTP/1.1" 200 267


Stopping web server


Set up a function to load a pre-trained ResNet50 model from Keras and a helper function to download an image from a URL, process it, and make a prediction. The output included the class predictions for the given images, showing the top predicted classes along with their descriptions and confidence scores.

#### Assignment 12.2

In [6]:
img_urls = [
    "https://source.unsplash.com/random/224x224?sig=1",
    "https://source.unsplash.com/random/224x224?sig=2",
    "https://source.unsplash.com/random/224x224?sig=3",
    "https://source.unsplash.com/random/224x224?sig=4",
    "https://source.unsplash.com/random/224x224?sig=5"
]


In [9]:
from io import BytesIO
from pathlib import Path
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import requests
import json

# Downloads and Loads the ResNetModel
model = ResNet50(weights='imagenet')

def load_img_url(img_url, target_size):
    r = requests.get(img_url)
    if r.status_code != 200:
        print('Could not download image')
        return None

    img = image.load_img(BytesIO(r.content), target_size=target_size)
    return img

def make_prediction(model, img_url, target_size=(224, 224), top=5):
    result = dict(src_img=img_url)
    img = load_img_url(img_url, target_size)
    if img is None:
        return result
    img = image.img_to_array(img)
    x = np.expand_dims(img, axis=0)
    x = preprocess_input(x)

    preds = model.predict(x)

    result['predictions'] = []

    for class_name, class_description, score in decode_predictions(preds, top=top)[0]:
        result['predictions'].append(dict(
            class_name=str(class_name),
            class_description=str(class_description),
            score=float(score)
        ))

    return result

# Define image URLs for testing
img_urls = [
    "https://source.unsplash.com/random/224x224?sig=1",
    "https://source.unsplash.com/random/224x224?sig=2",
    "https://source.unsplash.com/random/224x224?sig=3",
    "https://source.unsplash.com/random/224x224?sig=4",
    "https://source.unsplash.com/random/224x224?sig=5"
]

# Test the `make_prediction` function using img_urls
for img_url in img_urls:
    result = make_prediction(model, img_url)
    print('-------------------------------')
    print('')
    # Prints a pretty version of the result
    print(json.dumps(result, indent=2))
    print('-------------------------------')
    print('')


1/1 [==============================] - 1s 671ms/step
-------------------------------

{
  "src_img": "https://source.unsplash.com/random/224x224?sig=1",
  "predictions": [
    {
      "class_name": "n04366367",
      "class_description": "suspension_bridge",
      "score": 0.42213213443756104
    },
    {
      "class_name": "n04590129",
      "class_description": "window_shade",
      "score": 0.1882801502943039
    },
    {
      "class_name": "n03933933",
      "class_description": "pier",
      "score": 0.05842951685190201
    },
    {
      "class_name": "n04589890",
      "class_description": "window_screen",
      "score": 0.04089188203215599
    },
    {
      "class_name": "n03788195",
      "class_description": "mosque",
      "score": 0.03659722954034805
    }
  ]
}
-------------------------------

1/1 [==============================] - 0s 94ms/step
-------------------------------

{
  "src_img": "https://source.unsplash.com/random/224x224?sig=2",
  "predictions": [
    {
  

Built upon the previous setup by creating the make_prediction function that processes images from URLs, performs predictions using the ResNet50 model, and formats the results in a readable JSON format. This function was tested with a set of random image URLs, and the output displayed the model's predictions for each image, including the class names, descriptions, and confidence scores.

#### Assignment 12.3

In [ ]:
import falcon
import json
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import requests
from io import BytesIO

# Downloads and Loads the ResNetModel
model = ResNet50(weights='imagenet')

def load_img_url(img_url, target_size):
    r = requests.get(img_url)
    if r.status_code != 200:
        print('Could not download image')
        return None

    img = image.load_img(BytesIO(r.content), target_size=target_size)
    return img

def make_prediction(model, img_url, target_size=(224, 224), top=5):
    result = dict(src_img=img_url)
    img = load_img_url(img_url, target_size)
    if img is None:
        return result
    img = image.img_to_array(img)
    x = np.expand_dims(img, axis=0)
    x = preprocess_input(x)

    preds = model.predict(x)

    result['predictions'] = []

    for class_name, class_description, score in decode_predictions(preds, top=top)[0]:
        result['predictions'].append(dict(
            class_name=str(class_name),
            class_description=str(class_description),
            score=float(score)
        ))

    return result

class ClassifyResourceV2:
    def on_get(self, req, resp):
        img_url = req.get_param('img_url')
        if not img_url:
            resp.status = falcon.HTTP_400
            resp.media = {'error': 'img_url parameter is required'}
            return

        result = make_prediction(model, img_url)
        resp.media = result

app_v2 = falcon.App()
classify_v2 = ClassifyResourceV2()
app_v2.add_route('/classify', classify_v2)

# Function to run the server
def run_server(app):
    from wsgiref.simple_server import make_server
    httpd = make_server('127.0.0.1', 8000, app)
    print("Serving on port 8000...")
    httpd.serve_forever()

# Run the server
run_server(app_v2)


Serving on port 8000...


Combined the previous steps to implement a Falcon-based web service that handles GET requests. The ClassifyResourceV2 class was created to include the make_prediction function, and a Falcon app was set up to run on the local server. The server was successfully tested with client requests, and the output confirmed that the server correctly processed the images and returned predictions in JSON format.